# Predicting a smiling person

Data from https://github.com/lukas/smile.git

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Reshape
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [2]:
import smiledataset

In [3]:
# load data
X_train, y_train, X_test, y_test = smiledataset.load_data(block_size = 1)

In [4]:
num_classes = 2
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

img_rows, img_cols = X_train.shape[1:3]

In [5]:
channels = 3
X_train_ = np.zeros(shape = list(X_train.shape)+[channels])
X_test_ = np.zeros(shape = list(X_test.shape)+[channels])

In [6]:
for i in range(channels):
    X_train_[:,:,:,i] = X_train
    X_test_[:,:,:,i] = X_test

Normalization

In [7]:
X_train_ /= 255.0
X_test_ /= 255.0

VGG19 Feature generation

In [8]:
vgg_model = VGG19(include_top = False, input_shape = (img_rows, img_cols, channels))

In [9]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [10]:
X_train_vgg = vgg_model.predict(X_train_)
X_test_vgg = vgg_model.predict(X_test_)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_vgg, y_train, test_size = 0.2, random_state=42)

Build Model

In [12]:
from keras.layers import GlobalAveragePooling2D
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape = X_train.shape[1:]))
model.add(Dense(2, activation = 'sigmoid'))

In [13]:
# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Training
from keras.callbacks import ModelCheckpoint  
epochs = 30
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          epochs=epochs, batch_size=60, callbacks=[checkpointer], verbose=0)

Epoch 00000: val_loss improved from inf to 0.37455, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00001: val_loss improved from 0.37455 to 0.37386, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00002: val_loss improved from 0.37386 to 0.37334, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss improved from 0.37334 to 0.37333, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00005: val_loss did not improve
Epoch 00006: val_loss did not improve
Epoch 00007: val_loss improved from 0.37333 to 0.37206, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00008: val_loss improved from 0.37206 to 0.37117, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 00009: val_loss did not improve
Epoch 00010: val_loss did not improve
Epoch 00011: val_loss did not improve
Epoch 00012: val_loss improved from 0.37117 to 0.36949, saving model to saved_mo

In [16]:
predictions = model.predict(X_test_vgg)

In [18]:
np.sum(np.argmax(predictions, axis = 1) == np.argmax(y_test, axis = 1)) / predictions.shape[0]

0.81836528758829463